In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os
from tqdm import tqdm

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

# Find Best Sample

load hdf file

In [ ]:
hdfname = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss64_09Bn1000_2023-06-22_uniform-09b_n1000_r100_f40_l10/sam_lib.hdf5'
hdf = h5py.File(hdfname, 'r')
print(hdf.keys())

hc_ss_all = hdf['hc_ss'][...]
hc_bg_all = hdf['hc_bg'][...]
fobs = hdf['fobs'][:]
sample_params = hdf['sample_params'][...]
hdf.close()

nsamps, nfreqs, nreals, nloudest = [*hc_ss_all.shape]
print(f"{nsamps=}, {nfreqs=}, {nreals=}, {nloudest=}")


sort by 15yr match

In [ ]:
nsort, fidx, hc_ref15 = detstats.rank_samples(hc_ss_all, hc_bg_all, fobs, fidx=1, ret_all=True)
print(nsort[:5])


get parameters

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=None, seed=None)
param_names = pspace.param_names

In [ ]:
params = {}
for nn in nsort[:1]:
    print(f"sample {nn}:")
    for ii, target in enumerate(param_names):
        params[target] = sample_params[nn,ii]
        print(f"\t{target}={sample_params[nn,ii]}")
print(f"{params=}")

# Make new sample 219

In [ ]:
NREALS = 15
NLOUDEST = 100

In [ ]:
print(params)

In [ ]:
sam, hard = pspace.model_for_params(params=params, sam_shape=None)

In [ ]:
data = holo.librarian.run_model(sam, hard, nreals=NREALS, nloudest=NLOUDEST, nfreqs=40,
                              singles_flag=True, params_flag=True)

Get characteristic strain, mass, and distance

In [ ]:
hc_ss = data['hc_ss']
hc_bg = data['hc_bg']
fobs = data['fobs_cents']

mtot = data['sspar'][0] # g
mrat = data['sspar'][1] 
rz_f = data['sspar'][3]

mchirp = holo.utils.chirp_mass_mtmr(mtot, mrat)
dcom_f = holo.cosmo.comoving_distance(rz_f).to('cm').value # cm

print(mchirp.shape)

info = ['hc_ss: SS characteristic strain, indexed [F,R,L]',
        'hc_bg: characteristic strain minus SS, indexed [F,R]',
        'fobs: GW frequency bin centers in Hz, indexed [F]'
        'mchirp: chirp mass in g, indexed [F,R,L]',
        'dcom_f: final comoving distance in cm, indexed [F,R,L]',
        'F=frequency, R=realization, L=loudest SS']

Save to file

In [ ]:
fileloc = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss64_09Bn1000_2023-06-22_uniform-09b_n1000_r100_f40_l10'
filename = fileloc+'/bence_holodata_ss64_219'
np.savez(filename, hc_ss=hc_ss, hc_bg=hc_bg, fobs=fobs, mchirp=mchirp, dcom_f=dcom_f)